### Import bibliotek

In [4]:
import json
import requests
import os
from bs4 import BeautifulSoup

## Narzędzia


In [9]:
def extract(ancestor, selector=None, atribute=None, many = False):
    if selector:
        if many:
            if atribute:
                return [item[atribute].strip() for item in ancestor.select(selector)]
            return [item.text.strip() for item in ancestor.select(selector)]
        if atribute:
            try:
                return ancestor.select_one(selector)[atribute].text.strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if atribute:
        try:
            return ancestor[atribute]
        except TypeError:
            return None
    return ancestor.text.strip()
    

### Pobieranie ze strony Ceneo.pl opinii o konkretnym produkcie

In [8]:
with open("./headers.json","r") as jf:
    headers = json.load(jf)

In [ ]:
product_id = "84514582"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = requests.get(next_page)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
    for review in reviews:
        try:
            single_review = {
                key:extract(review,value)
                for key, value in review_scheme.items()
            }
            all_reviews.append(single_review)

        except (AttributeError, TypeError):
            pass
    try:
        next_page = "htttps://www.ceneo.pl" + extract(page_dom, "a.pagination_next","href")
    except TypeError:
        next_page = None
print(all_reviews)

    

In [18]:
review_scheme = {
                "review_id": (None,'data-entry-id'),
                "author": ("span.user-post__author-name"),
                "recomendation": ("span.user-post__author-recomendation > em"),
                "stars": ("span.user-post__score-count"),
                "content" : ("div.user-post__text"),
                "pros": ("div.review-feature__item--positive",None,True),
                "cons": ("div.review-feature__item--negative",None,True),
                "likes" :("button.vote-yes > span"),
                "dislikes" : ("button.vote-no > span"),
                "publish_date" : ("span.user-post__published > time:nth-child(1)['datetime']"),
                "purchase_date" :("span.user-post__published > time:nth-child(2)['datetime']"),
            }

Zapisanie wszystkich opinii w bazie danych

In [5]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")
    

In [1]:
with open(f"./opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_reviews, jf, indent=4, ensure_ascii=False)

NameError: name 'product_id' is not defined